In [ ]:
!pip install ipython-autotime
# !pip install -U scikit-learn
%load_ext autotime

time: 1.71 ms (started: 2021-01-17 15:00:24 +00:00)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 5.99 ms (started: 2021-01-17 15:00:24 +00:00)


In [ ]:
!pwd
import os
os.chdir('/content/drive/My Drive/ML/ml-project-master/src/')
!pwd

/content
/content/drive/My Drive/ML/ml-project-master/src
time: 236 ms (started: 2021-01-17 15:00:24 +00:00)


In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import tensorflow as tf

from utility import *

time: 2.25 s (started: 2021-01-17 15:00:24 +00:00)


# Import Dataset

In [ ]:
Xtrain, Xtest, ytrain, ytest = load_monk("3")
print(Xtrain.shape)
print(Xtest.shape)
print(ytrain.shape)
print(ytest.shape)

(124, 17)
(430, 17)
(124, 1)
(430, 1)
time: 20.3 ms (started: 2021-01-17 15:00:27 +00:00)


In [ ]:
# X, y = fetch_openml('monks-problems-3', return_X_y=True)
# y = y.astype(np.float32)

# X = OneHotEncoder(sparse=False).fit_transform(X)
# y = y.reshape(y.shape[0], 1)

time: 1.39 ms (started: 2021-01-17 15:00:27 +00:00)


# Grid Search - Gradient Descent

In [ ]:
best_results_filename = "best_results_monk3.txt"
single_grid_results_filename = 'grid_results_monk3.1.csv'

time: 3.2 ms (started: 2021-01-17 15:00:30 +00:00)


In [ ]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import SGD

# L2 = tf.keras.regularizers.L2(l2=0.02)

# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.01, momentum=0, neurons=1, init_mode='uniform', activation='relu'):
  # create model
  model = Sequential()
  model.add(layers.InputLayer(input_shape=(17)))
  model.add(Dense(neurons, activation=activation, kernel_initializer=init_mode))
  model.add(Dense(1, activation='sigmoid', kernel_initializer=init_mode))
  optimizer = SGD(
      learning_rate=learn_rate, momentum=momentum, nesterov=False)
  # Compile model
  model.compile(optimizer=optimizer,
              loss=tf.keras.losses.MSE,
              metrics='accuracy')
  return model

BATCH_SIZE = len(Xtrain)

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [32, 64, BATCH_SIZE]
epochs = [1000]
learn_rate = [0.001, 0.01, 0.1]
momentum = [0.0, 0.2, 0.6, 0.8, 0.9]
neurons = [4, 5]
init_mode = ['glorot_uniform']
activation = ['relu', 'tanh', 'sigmoid']

# momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
# neurons = [2, 3, 4, 5]
# init_mode = ['uniform', 'normal', 'zero', 'glorot_uniform']
# optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

param_grid = dict(batch_size=batch_size, epochs=epochs, learn_rate=learn_rate, momentum=momentum, neurons=neurons, init_mode=init_mode, activation=activation)#, optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(Xtrain, ytrain)


In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


grid_values_string = f'Grid on: batch_size: {batch_size}, epochs: {epochs}, learn_rate: {learn_rate}, momentum: {momentum}, init_mode = {init_mode} \n'
grid_results_string = f'Best: {grid_result.best_score_} using {grid_result.best_params_} \n'
grid_correspondant_parameter_name = f'Results in: {single_grid_results_filename} \n'
# Open a file with access mode 'a'
with open(best_results_filename, "a") as file_object:
    # Append 'hello' at the end of file
    file_object.write(grid_correspondant_parameter_name)
    file_object.write(grid_values_string)
    file_object.write(grid_results_string)

In [ ]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
import csv

with open(single_grid_results_filename, mode='w') as grid:
    grid = csv.writer(grid, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for mean, stdev, param in zip(means, stds, params):
      grid.writerow([mean, stdev, param])

In [ ]:
# from sklearn.externals import joblib

# # #save your model or results
# # joblib.dump(grid_result, 'gs_object_monk3.pkl')
# # #load your model for further usage
# # boh = joblib.load("gs_object_monk3.pkl")

# # joblib.dump(grid.best_estimator_, 'gs_best_estimator_monk3.pkl', compress = 1)
# # filename = 'finalized_model.sav'
# # pickle.dump(model, open(filename, 'wb'))

# Grid Search - Gradient Descent with regularization

In [ ]:
best_results_filename = "best_results_monk3.txt"
single_grid_results_filename = 'grid_results_monk3.2.csv'

time: 1.19 ms (started: 2021-01-17 14:10:33 +00:00)


In [ ]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import L2

# L2 = tf.keras.regularizers.L2(l2=0.02)

# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.01, momentum=0, neurons=1, init_mode='uniform', activation='relu', reg_L2 = 0.0):
  # create model
  model = Sequential()
  model.add(layers.InputLayer(input_shape=(17)))
  model.add(Dense(neurons, activation=activation, kernel_initializer= init_mode, kernel_regularizer= L2(reg_L2) ))
  model.add(Dense(1, activation='sigmoid', kernel_initializer=init_mode, kernel_regularizer= L2(reg_L2)))
  optimizer = SGD(
      learning_rate=learn_rate, momentum=momentum, nesterov=False)
  # Compile model
  model.compile(optimizer=optimizer,
              loss=tf.keras.losses.MSE,
              metrics='accuracy')
  return model

BATCH_SIZE = len(Xtrain)

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [32, 64, BATCH_SIZE]
epochs = [1000]
learn_rate = [0.1, 0.001, 0.01, 0.1]
momentum = [0.0, 0.2, 0.6, 0.8, 0.9]
neurons = [4, 5]
init_mode = ['glorot_uniform']
activation = ['tanh'] #['relu', 'tanh', 'sigmoid']
#L2_list = [0.0, 0.1, 0.2, 0.01]

# init_mode = ['uniform', 'normal', 'zero', 'glorot_uniform']
# optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

param_grid = dict(batch_size=batch_size, epochs=epochs, learn_rate=learn_rate, momentum=momentum, neurons=neurons, init_mode=init_mode, activation=activation, reg_L2 = L2_list )#, optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(Xtrain, ytrain)


KeyboardInterrupt: ignored

time: 41min 9s (started: 2021-01-17 14:10:33 +00:00)


In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


grid_values_string = f'Grid on: batch_size: {batch_size}, epochs: {epochs}, learn_rate: {learn_rate}, momentum: {momentum}, init_mode = {init_mode} \n'
grid_results_string = f'Best: {grid_result.best_score_} using {grid_result.best_params_} \n'
grid_correspondant_parameter_name = f'Results in: {single_grid_results_filename} \n'
# Open a file with access mode 'a'
with open(best_results_filename, "a") as file_object:
    # Append 'hello' at the end of file
    file_object.write(grid_correspondant_parameter_name)
    file_object.write(grid_values_string)
    file_object.write(grid_results_string)

In [ ]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
import csv

with open(single_grid_results_filename, mode='w') as grid:
    grid = csv.writer(grid, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for mean, stdev, param in zip(means, stds, params):
      grid.writerow([mean, stdev, param])

In [ ]:
# from sklearn.externals import joblib

# # #save your model or results
# # joblib.dump(grid_result, 'gs_object_monk3.pkl')
# # #load your model for further usage
# # boh = joblib.load("gs_object_monk3.pkl")

# # joblib.dump(grid.best_estimator_, 'gs_best_estimator_monk3.pkl', compress = 1)
# # filename = 'finalized_model.sav'
# # pickle.dump(model, open(filename, 'wb'))